In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import json 
# Lead chunks.json into a dictionary
with open('chunks.json', 'r') as f:
    chunks = json.load(f)

In [ ]:
#Convert dict to a tuple of (game, review)
reviews = []
for game in chunks:
    for review in chunks[game]:
        reviews.append((game, review))


In [ ]:
# Create a list for reviews and a list for games
review_list = []
game_list = []
for review in reviews:
    game_list.append(review[0])
    review_list.append(review[1])


In [ ]:
# Convert each game to a dictionary of gameDict["game"] = [game]
gameDictList = []
for game in game_list:
    gameDict = {}
    gameDict["game"] = [game]
    gameDictList.append(gameDict)
gameDictList

In [2]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [4]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [5]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [6]:
#docsearch = Pinecone.from_texts(review_list, embeddings, metadatas=gameDictList, index_name=index_name) #Initial population !!
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [7]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [8]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
docs = docsearch.similarity_search_with_score(query, k=5)


In [ ]:
docs2 = docsearch.similarity_search(query, k=5)
docs2

In [ ]:
game_scores = {}

for doc, score in docs:
    game_name = doc.metadata['game'][0]
    if game_name in game_scores:
        game_scores[game_name].append(score)
    else:
        game_scores[game_name] = [score]


In [ ]:
games = []
for i in range(len(docs)):
    games.append(docs[i][0].metadata["game"][0])
games

In [ ]:
game_name = games[0]

In [ ]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
game_docs = docsearch.similarity_search(query, k=5, filter={"game": game_name})

In [ ]:
game_docs

In [ ]:
query = "Reccommend this game to me, If you wouldn't reccommend it always start your answer wtih a No. Make relations to my query: " + query
chain.run(input_documents=game_docs, question=query)

In [ ]:
for i in range (len(game_docs)):
    print(i)

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

def recommend_game(query, game_name):
    game_docs = docsearch.similarity_search(query, k=5, filter={"game": game_name})
    game_docs_content = " ".join(doc.page_content for doc in game_docs)
    game_docs_metadata = "Metadata: " + str(game_docs[0].metadata) if game_docs else ""
    messages = [
        SystemMessage(content="Unless the game is completely unrelated to the input recommend it and start your answer with 'Yes'. If you definitely can't recommend it start with No. Don't say the game name in your response. Describe the game in your recommendation. Don't write in the first person."),
        HumanMessage(content=game_docs_content + " " + game_docs_metadata)
    ]
    answer = chat(messages)
    return answer.content  # Assuming you want the content of the AIMessage


In [15]:
game_docs = docsearch.similarity_search("I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic.", k=5, filter={"game": "Terraria"})


In [20]:
recommend_game("I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic.", "Terraria")

'Yes, I highly recommend Terraria. It is a 2D sandbox RPG with platformer elements that offers an incredible amount of content and replayability. The game revolves around defeating bosses to progress and unlock new content, but the player has high agency and can play and progress at their own pace. With over 4000 items in the game, every playthrough feels unique and not repetitive. It offers both a casual and challenging experience for players. The game is only 200mb but offers hundreds of hours worth of content. The game has some cons such as inventory management and a difficulty spike after defeating the boss in the Underworld, but overall it is an amazing experience from beginning to end. The game has great music, easy multiplayer setup, and runs well on most computers. Terraria is very inexpensive and worth every penny.'

In [21]:
import requests
from bs4 import BeautifulSoup

def get_steam_url(game_name):
    game_name = game_name.replace(' ', '+')
    search_url = f"https://store.steampowered.com/search/?term={game_name}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    game_link = soup.find('a', {'class': 'search_result_row ds_collapse_flag'})
    if game_link:
        return game_link['href']
    else:
        return None

https://store.steampowered.com/app/1091500/Cyberpunk_2077/?snr=1_7_7_151_150_1


In [22]:
# Read chunks.json and test the function on each game
import json

with open('chunks.json', 'r') as f:
    chunks = json.load(f)

# gamename is the key, chunk is the value
for gamename, chunk in chunks.items():
    print(gamename)
    print(get_steam_url(gamename))
    print()

Counter-Strike: Global Offensive
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_151_150_1

Far Cry® 6
https://store.steampowered.com/app/2369390/Far_Cry_6/?snr=1_7_7_151_150_1

Team Fortress 2
https://store.steampowered.com/app/440/Team_Fortress_2/?snr=1_7_7_151_150_1

Forza Horizon 5
https://store.steampowered.com/app/1551360/Forza_Horizon_5/?snr=1_7_7_151_150_1

Crusader Kings III
https://store.steampowered.com/app/1158310/Crusader_Kings_III/?snr=1_7_7_151_150_1

EA SPORTS™ FIFA 23
https://store.steampowered.com/app/1811260/EA_SPORTS_FIFA_23/?snr=1_7_7_151_150_1

Hearts of Iron IV
https://store.steampowered.com/app/394360/Hearts_of_Iron_IV/?snr=1_7_7_151_150_1

Call of Duty®: Modern Warfare® II
https://store.steampowered.com/app/1938090/Call_of_Duty_Modern_Warfare_II/?snr=1_7_7_151_150_1

Lost Ark
https://store.steampowered.com/app/1599340/Lost_Ark/?snr=1_7_7_151_150_1

War Thunder
https://store.steampowered.com/app/236390/War_Thunder/?snr=1_7_7_151_

KeyboardInterrupt: 

In [23]:
print(get_steam_url("The Witcher\u00ae 3: Wild Hunt"))

https://store.steampowered.com/app/292030/The_Witcher_3_Wild_Hunt/?snr=1_7_7_151_150_1
